<a href="https://colab.research.google.com/github/woodyx218/Deep-Learning-with-GDP-Pytorch/blob/master/GDP_NN_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Differentailly Private Deep Learning accounted by Gaussian differential privacy

### This is a tutorial of how to train deep neural networks with differential privacy, a mathematically rigorous privacy definition that defends against privacy leakage, e.g. the [membership inference attack](https://github.com/tensorflow/privacy/tree/master/tensorflow_privacy/privacy/membership_inference_attack).

### Our tutorial is based on the Pytorch library, [**Opacus**](https://github.com/pytorch/opacus). First we install Opacus.

In [1]:
!pip install opacus

     |████████████████████████████████| 102kB 5.8MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


### We also need the privacy accountants including our GDP accountant

In [3]:
!git clone https://github.com/woodyx218/Deep-Learning-with-GDP-Pytorch.git

Cloning into 'Deep-Learning-with-GDP-Pytorch'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 33 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


In [4]:
cd /content/Deep-Learning-with-GDP-Pytorch

/content/Deep-Learning-with-GDP-Pytorch


### Import necessary packages.

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from opacus import PrivacyEngine
from torchvision import datasets, transforms
from tqdm import tqdm
from torch.utils.data import DataLoader

### We use MNIST here, similar to 
https://github.com/pytorch/opacus/blob/master/examples/mnist.py

In [8]:
batch_size=250

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("../data",train=True,download=True,
                   transform=transforms.Compose(
                       [transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081)),])
                   ), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("../data",train=False,
                   transform=transforms.Compose(
                       [transforms.ToTensor(),
                        transforms.Normalize((0.1307), (0.3081)),])
                   ), batch_size=1000, shuffle=False)


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### We define a simple CNN, almost same as [Tensorflow Privacy example](https://github.com/tensorflow/privacy/blob/master/tutorials/Classification_Privacy.ipynb) and [Opacus example](https://github.com/pytorch/opacus/blob/master/examples/mnist.py), but we use **tanh** activation instead of **relu** for better performance (see https://arxiv.org/abs/2007.14191).

In [9]:
class SampleConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = nn.Conv2d(16, 32, 4, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        # x of shape [B, 1, 28, 28]
        x = F.tanh(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.tanh(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.tanh(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x

    def name(self):
        return "SampleConvNet"

device = torch.device('cuda')
cnn_DP=SampleConvNet().to(device)

### Now that we have the data preprocessed and the model defined, we move on to set the training process for the optimizer. We define a class to include the hyperparameters.

In [10]:
class Args:
    epochs=15 # number of epochs
    n_runs=1 # allow multiple independent run so you can take average of accuracy
    lr=0.25 # learning rate
    sigma=1.3 # noise multiplier in the DP optimizer, necessary to set to non-zero for DP
    max_per_sample_grad_norm=1.5  # clipping norm to guarantee the sensitivity of gradients; necessary to set to non-infinity for DP
    delta=1e-5 # delta in (epsilon, delta)-DP, need to be smaller than 1/sample size
    
    disable_dp=False  
    # if True, then this is training with no clipping (hence not DP); 
    # if False, and if sigma not zero, then DP; else, this is non-DP training with clipping
    
args = Args()

### Next, we define the training function and attach any regular optimizer (SGD, Adagrad, Adam) to the **privacy engine** (which automatically turns optimizer into DP-optimizer, and calculates your privacy risk on the fly).

In [11]:
def epoch(loader, model, opt=None):
    total_loss, total_err = 0.,0.
    if opt:
      model.train()
    else:
      model.eval()
    for X,y in loader:
        X,y = X.to(device), y.to(device)
        yp = model(X)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += (yp.max(dim=1)[1] != y).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

opt_DP = optim.SGD(cnn_DP.parameters(), lr=args.lr)

# only this privacy engine is from Opacus
privacy_engine = PrivacyEngine(
    cnn_DP,
    batch_size=batch_size,
    sample_size=len(train_loader.dataset),
    alphas=[1 + x / 10.0 for x in range(1, 100)] + list(range(12, 64)),
    noise_multiplier=args.sigma,
    max_grad_norm=args.max_per_sample_grad_norm,
)
privacy_engine.attach(opt_DP)
    
print(' '*10,'Train Accuracy , Train Cross-Entropy Loss, Test Accuracy , Test Cross-Entropy Loss, ')
for Epoch in range(args.epochs):
    train_err_DP, train_loss_DP = epoch(train_loader, cnn_DP, opt_DP)
    test_err_DP, test_loss_DP = epoch(test_loader, cnn_DP)
    print('Epoch',Epoch+1,'--> DP:',*("{:.6f}".format(i) for i in (1-train_err_DP, train_loss_DP, 1-test_err_DP, test_loss_DP)), sep="\t")
    epsilon, best_alpha = privacy_engine.get_privacy_spent()
    print(f" | (ε = {epsilon:.2f}, δ = {privacy_engine.target_delta}) for α = {best_alpha}")

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:518: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  "The sample rate will be defined from ``batch_size`` and ``sample_size``."
/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  wa

           Train Accuracy , Train Cross-Entropy Loss, Test Accuracy , Test Cross-Entropy Loss, 
Epoch	1	--> DP:	0.748200	0.914032	0.896900	0.352260
 | (ε = 0.85, δ = 1e-06) for α = 18.0
Epoch	2	--> DP:	0.912100	0.293044	0.930300	0.229040
 | (ε = 0.89, δ = 1e-06) for α = 18.0
Epoch	3	--> DP:	0.932617	0.222917	0.944700	0.188370
 | (ε = 0.93, δ = 1e-06) for α = 18.0
Epoch	4	--> DP:	0.942767	0.189695	0.948000	0.167144
 | (ε = 0.96, δ = 1e-06) for α = 18.0
Epoch	5	--> DP:	0.948000	0.174212	0.952300	0.160660
 | (ε = 1.00, δ = 1e-06) for α = 18.0
Epoch	6	--> DP:	0.951833	0.165832	0.954100	0.154810
 | (ε = 1.04, δ = 1e-06) for α = 18.0
Epoch	7	--> DP:	0.953317	0.160666	0.956400	0.148192
 | (ε = 1.08, δ = 1e-06) for α = 18.0
Epoch	8	--> DP:	0.955383	0.152665	0.957000	0.146496
 | (ε = 1.11, δ = 1e-06) for α = 17.0
Epoch	9	--> DP:	0.957617	0.145775	0.960600	0.136925
 | (ε = 1.14, δ = 1e-06) for α = 17.0
Epoch	10	--> DP:	0.958733	0.142418	0.961700	0.134736
 | (ε = 1.17, δ = 1e-06) for α = 17.0
Epo

### There exist many privacy accountants that give you an upper bound on epsilon. The most popular ones are [Moments Accountant](https://arxiv.org/abs/1607.00133) (MA) and Gaussian DP (GDP; promoted by this paper).

### There are many variants of MA. E.g. in the above, Opacus uses a version of MA that gives epsilon = 1.32. In below, Tensorflow Privacy uses another MA that gives epsilon = 0.94 and our GDP accountant gives epsilon = 0.82. Hence GDP is the tightest privacy guarantee.

In [6]:
!pip install tensorflow-privacy
from gdp_accountant import compute_epsP, compute_epsilon # which uses tensorflow-privacy package
print('eps_MA=%.6g' %(compute_epsilon(15,1.3,60000,250,1e-5)))
print('eps_GDP=%.6g' %(compute_epsP(15,1.3,60000,250,1e-5)))

     |████████████████████████████████| 194kB 13.2MB/s 
eps_MA=0.9422
eps_GDP=0.823696


## Some Tips:

1. Larger epsilon and delta means model is less private
2. Fixing other hyperparameters, more iterations/epochs less private
3. Learning rate and clipping norm do not affect DP but affect convergence
4. Iteration, sigma, batch size and sample size affect DP
5. One set of training corresponds to infinitely many (epsilon,delta) pairs, usually we choose delta = 1/sample size
6. Practionally, one tunes clipping norm first with sigma = 0, then fix clipping norm and tune sigma, under pre-specified privacy budget (e.g. epsilon < 8)
